In [11]:
!pip install --quiet --upgrade openai tenacity pandas ipywidgets


In [12]:
import os, time, json, math
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

from llm_utils import chat_completion, TOKENS_USED,client
from typing import List, Dict



In [13]:
# === 2. 定义 UI 元素 ===
desc_input = widgets.Textarea(
    value="An aluminum gear with 20 teeth and a central bore.",
    placeholder="Describe the part here...",
    description="Part:",
    layout=widgets.Layout(width="100%", height="100px")
)

material_selector = widgets.Dropdown(
    options=['aluminum', 'steel', 'brass', 'titanium', 'plastic'],
    value='aluminum',
    description='Material:',
    layout=widgets.Layout(width='30%')
)


generate_button = widgets.Button(
    description=" Generate CNC Plan",
    button_style="success",
    layout=widgets.Layout(width="30%", margin="10px 0")
)

output_area = widgets.Output()


In [14]:
# === Global machining limits ======================================
MATERIAL_LIMITS = {
    "aluminum": {"rpm": (3000, 12000), "feed": (800, 1500)},
    "steel":    {"rpm": (500, 1500),   "feed": (100, 300)},
    "brass":    {"rpm": (1500, 6000),  "feed": (400, 800)},
    "titanium": {"rpm": (100, 500),    "feed": (50, 200)},
    "plastic":  {"rpm": (2000, 8000),  "feed": (500, 1500)},
}

def validate_plan(df, material: str):
    """
    Add 'RPM Valid' and 'Feed Valid' columns based on MATERIAL_LIMITS.
    """
    limits = MATERIAL_LIMITS.get(material)
    if limits is None:
        # Unknown material -> mark everything invalid but keep running
        df["RPM Valid"]  = False
        df["Feed Valid"] = False
        return df

    rpm_min, rpm_max   = limits["rpm"]
    feed_min, feed_max = limits["feed"]

    df["RPM Valid"]  = df["Spindle Speed (RPM)"].between(rpm_min, rpm_max)
    df["Feed Valid"] = df["Feed Rate (mm/min)"].between(feed_min, feed_max)
    return df



In [15]:
def get_outline(part: str, material: str) -> List[Dict[str, str]]:
    """
    Ask the LLM for a high-level process outline and
    return a list of {"step": "..."} dictionaries.
    """
    system_msg = {
        "role": "system",
        "content": (
            "You are a CNC process planner. "
            "Return ONLY a JSON array. Each item must have the key 'step'."
        )
    }
    user_msg = {
        "role": "user",
        "content": (
            f"The part is: {part}\n"
            f"The material is: {material}\n"
            "List the high-level manufacturing steps needed to machine this part."
        )
    }

    try:
        response = chat_completion(
            messages=[system_msg, user_msg],
            verbose=False
        )
        return json.loads(response)
    except Exception as e:
        print(" get_outline failed:", e)
        return []
    
    
def get_detail(outline: List[Dict[str, str]] | str,
               part: str,
               material: str) -> str:
    """
    Enrich each step with tool, operation, rpm and feed.
    Returns raw JSON string.
    """

    # --- 1. Convert outline list to bullet list text -----------------
    outline_text = (
        "\n".join(f"- {s['step']}" for s in outline if 'step' in s)
        if isinstance(outline, list) else outline
    )

    # --- 2. Fetch material-specific limits --------------------------
    limits = MATERIAL_LIMITS.get(material)
    if limits is None:
        # fallback to a safe generic range
        limits = {"rpm": (500, 5000), "feed": (100, 1000)}

    rpm_min, rpm_max   = limits["rpm"]
    feed_min, feed_max = limits["feed"]

    # create constraint sentence, e.g.:
    # "For titanium, spindle speed must be 100-500 rpm and feed 50-200 mm/min."
    material_constraints = (
        f"For {material}, spindle speed must be {rpm_min}-{rpm_max} rpm "
        f"and feed rate {feed_min}-{feed_max} mm/min."
    )

    # --- 3. Build messages list for chat_completion -----------------
    system_msg = {
        "role": "system",
        "content": "You are a CNC process planner."
    }

    user_msg = {
        "role": "user",
        "content": (
            f"The part is: {part}\n"
            f"The material is: {material}\n"
            f"Here is the outline of steps:\n{outline_text}\n\n"
            "For EACH step output an object with: step, tool, operation, rpm, feed.\n"
            f"{material_constraints}\n"
            "For non-machining steps (e.g. selection, inspection) set rpm=0 and feed=0.\n\n"
            "Return ONLY a JSON array. No explanations. No markdown."
        )
    }

    # --- 4. Call the LLM -------------------------------------------
    return chat_completion(
        messages=[system_msg, user_msg],
        verbose=False
    )


In [16]:
def parse_llm_output(raw_json):
    data = json.loads(raw_json)
    df = pd.DataFrame(data)
    df.rename(columns={
        "rpm": "Spindle Speed (RPM)",
        "feed": "Feed Rate (mm/min)"
    }, inplace=True)
    return df


In [17]:
def display_plan_table(df):
    def highlight_invalid(v):
        return "background-color:#FFD2D2" if v is False else ""
    styled = df.style.map(highlight_invalid, subset=["RPM Valid","Feed Valid"])
    display(Markdown("### CNC Process Plan"))
    display(styled)


In [18]:
# === 工艺总结 ===
def reflect_summary(raw_json: str, validated_df: pd.DataFrame):
    num_steps = len(validated_df)
    num_invalid_rpm   = (~validated_df["RPM Valid"]).sum()
    num_invalid_feed  = (~validated_df["Feed Valid"]).sum()

    comment = (
        "### Reflection Summary\n"
        f"- **Total Steps Generated**: {num_steps}\n"
        f"- **Invalid Spindle Speeds**: {num_invalid_rpm} step(s)\n"
        f"- **Invalid Feed Rates**: {num_invalid_feed} step(s)\n"
    )

    if num_invalid_rpm or num_invalid_feed:
        comment += "- ** Human Oversight Needed**:\n"
        if num_invalid_rpm:
            comment += "  - Some spindle speeds out of range.\n"
        if num_invalid_feed:
            comment += "  - Some feed rates out of range.\n"
    else:
        comment += "- All parameters are within expected machining constraints.\n"

    # 统计 token
    from llm_utils import TOKENS_USED
    comment += f"- **Tokens used so far**: {TOKENS_USED}\n"

    display(Markdown(comment))


In [19]:
# === 4. 按钮点击逻辑 ===
def on_generate_clicked(b):
    with output_area:
        output_area.clear_output()

        part = desc_input.value.strip()
        material = material_selector.value

        if not part:
            print("Please enter the part description.")
            return

        # --- ① 先取高阶大纲 ---------------------------------
        outline = get_outline(part, material)
        if not outline:
            print("Failed to retrieve outline. Please check your network connection or API key.")
            return

        # --- ② 再根据大纲生成带参数的完整 JSON ---------------
        raw_json = get_detail(outline, part, material)

        # --- ③ 解析 LLM JSON → DataFrame -------------------
        df_full = parse_llm_output(raw_json).reset_index(drop=True)
        if df_full.empty:
            print("JSON parsing failed, raw output:\n", raw_json)
            return

        # --- ④ 全流程校验并展示 ------------------------------
        df_full_valid = validate_plan(df_full, material)
        display(Markdown("### Full Process Plan (incl. non-machining)"))
        display_plan_table(df_full_valid)

        # --- ⑤ 过滤仅含切削参数的步骤 -------------------------
        df_cut = df_full[
            df_full["Spindle Speed (RPM)"].gt(0) &
            df_full["Feed Rate (mm/min)"].gt(0)
        ].reset_index(drop=True)

        df_cut_valid = validate_plan(df_cut, material)
        display(Markdown("### Machining-only Plan"))
        display_plan_table(df_cut_valid)

        # --- ⑥ 反思摘要（基于完整流程） -----------------------
        reflect_summary(raw_json, df_full_valid)
        
        

In [20]:
# === 5. 按钮事件绑定 ===
generate_button.on_click(on_generate_clicked)

# === 6. 显示UI（必须放最后） ===
output_area = widgets.Output()
display(
    Markdown("## CNC Process Planner"),
    desc_input,
    material_selector,
    generate_button,
    output_area
)   


## CNC Process Planner

Textarea(value='An aluminum gear with 20 teeth and a central bore.', description='Part:', layout=Layout(height…

Dropdown(description='Material:', layout=Layout(width='30%'), options=('aluminum', 'steel', 'brass', 'titanium…

Button(button_style='success', description=' Generate CNC Plan', layout=Layout(margin='10px 0', width='30%'), …

Output()